작사가 인공지능 만들기
========

LSTM 모델과 셰익스피어 데이터셋을 사용해 간단한 작사가 인공지능을 만들어 보기

## 1. 데이터 읽어오기

In [1]:
# 사용할 라이브러리 불러오기
import glob
import os
import re 
import numpy as np
import tensorflow as tf

In [2]:
# 파일에서 데이터 읽어오기
txt_file_path = os.getenv('HOME')+'/aiffel/Exploration/E4/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])  # 앞에서부터 3라인만 화면에 출력

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


## 2. 데이터 정제

  
preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하기    
     
추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거    
너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수 있기 때문에      
문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기     

In [3]:
## 토큰화 하기 위한 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.'!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.'!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("I know you'll do the best you can."))

<start> i know you ' ll do the best you can . <end>


1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.'!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 \<start>, 끝에는 \<end>를 추가합니다     

In [5]:
## 정제 데이터 구축

corpus = [] # 정제된 문장을 모음

for sentence in raw_corpus:
    # 길이가 0 이면 건너 뜀 
    if len(sentence) == 0 : continue
    #if len(sentence.split()) > 15 : continue
        
    # 정제를 하고 담아주기
    preprocessed_sentence = preprocess_sentence(sentence)
    # 정제 문장이 이전 문장과 같으면 건너 뜀
    if len(corpus) != 0 and preprocessed_sentence == corpus[-1]: continue
    corpus.append(preprocessed_sentence)
        
print("데이터 크기:", len(corpus))
print("Examples:\n", corpus[:10])  # 정제된 결과를 10개만 확인

데이터 크기: 171615
Examples:
 ['<start> there must be some kind of way outta here <end>', '<start> said the joker to the thief <end>', "<start> there ' s too much confusion <end>", "<start> i can ' t get no relief business men , they drink my wine <end>", '<start> plowman dig my earth <end>', '<start> none were level on the mind <end>', '<start> nobody up at his word <end>', '<start> hey , hey no reason to get excited <end>', '<start> the thief he kindly spoke <end>', '<start> there are many here among us <end>']


## 3. 평가 데이터셋 분리

     
tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하기    
단어장의 크기는 12,000 이상 으로 설정하기    
총 데이터의 20% 를 평가 데이터셋으로 사용하기    


### tokenize() 함수로 데이터를 Tensor로 변환

In [6]:
## 토큰화 하기
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용

def tokenize(corpus):
    # 12000단어 이상을 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 14000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  65 279 ...   0   0   0]
 [  2 108   7 ...   0   0   0]
 [  2  65   4 ...   0   0   0]
 ...
 [  2  76  47 ... 872   3   0]
 [  2  50   5 ...   0   0   0]
 [  2  14 632 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f9317317250>


- tf.keras.preprocessing.text.Tokenizer 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해줍니다. 이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭합니다. 우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것입니다.


- tf.keras.preprocessing.text.Tokenizer()    
    https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
- tf.keras.preprocessing.sequence.pad_sequences()     
    https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [7]:
print(tensor.shape)
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:3, :10])

(171615, 15)
[[   2   65  279   28   99  524   20   85  790   94]
 [   2  108    7 6439   11    7 2507    3    0    0]
 [   2   65    4   17  103  181 2941    3    0    0]]


In [8]:
# 단어 사전이 어떻게 구축되었는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : '
5 : ,
6 : i
7 : the
8 : you
9 : and
10 : a


### sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리

In [9]:
## 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하기

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  65 279  28  99 524  20  85 790  94   3   0   0   0]
[ 65 279  28  99 524  20  85 790  94   3   0   0   0   0]


In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (137292, 14)
Target Train: (137292, 14)


- 데이터 정제 과정을 잘 거치면 학습 데이터 개수가 124960보다 작을 것이라고 하였습니다.      
    하지만 데이터 정제를 제가 한 것 이외에 어떻게 더 해야할지 잘 모르겠습니다. 

## 4. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하기

In [13]:
class lyricist(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = lyricist(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
4291/4291 [==============================] - 364s 83ms/step - loss: 3.6684
Epoch 2/10
4291/4291 [==============================] - 360s 84ms/step - loss: 2.9248
Epoch 3/10
4291/4291 [==============================] - 357s 83ms/step - loss: 2.6396
Epoch 4/10
4291/4291 [==============================] - 358s 84ms/step - loss: 2.3806
Epoch 5/10
4291/4291 [==============================] - 355s 83ms/step - loss: 2.1538
Epoch 6/10
4291/4291 [==============================] - 354s 82ms/step - loss: 1.9472
Epoch 7/10
4291/4291 [==============================] - 357s 83ms/step - loss: 1.7725
Epoch 8/10
4291/4291 [==============================] - 356s 83ms/step - loss: 1.6105
Epoch 9/10
4291/4291 [==============================] - 354s 83ms/step - loss: 1.4701
Epoch 10/10
4291/4291 [==============================] - 354s 83ms/step - loss: 1.3554


- loss 값은 최종 1.36으로 2.2 보다 낮은 것을 볼 수 있다. 

### 평가하기

In [14]:
def generate_lyric(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love craig craig platter platter she she olivia olivia olivia womb womb womb territory territory gangster gangster bumped '

In [16]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> hey you", max_len=20)

'<start> hey you ooh ooh tripping tripping fronts sprayed desperado desperado affect booth booth rodeo kool kool humpty humpty humpty '

In [17]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> my first", max_len=20)

'<start> my first homesick homesick gentleman gentleman gingerbread gingerbread gingerbread chicken learns learns learns learns learns learns boring boring boring '

In [18]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> i want", max_len=10)

'<start> i want homesick homesick hottest hottest hottest whole whole '

In [19]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> i love", max_len=10)

'<start> i love craig craig platter platter she she olivia '

In [20]:
generate_lyric(lyricist, tokenizer, init_sentence="<start> my love", max_len=10)

'<start> my love dion dion opening willing willing willing willing '

- 나온 가사들을 보면 중복 단어가 많은 것을 볼 수 있었다.    
    그래도 노래 가사인 것을 생각하면 괜찮다고 생각한다. 